# PixelCNN Denoiser

## Set up variables for training

In [ ]:
import models.PixelVAEDenoiser as PAE
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader

learning_rate = 0.001
batchsize = 128
epochs = 5
# loss_fn = torch.nn.MSELoss()
loss_fn = torch.nn.BCEWithLogitsLoss()
device='cuda'

# Model hyperparameters
features = 60
rec_field = 256
encdec_channels = [16, 64, 128]
upsamples = [2, 2, 2]
latent_space_dim = 32
embedding_dim = 64

encoder = PAE.Encoder(input_size = (features, rec_field), channel_sizes = encdec_channels, upsamples = upsamples, zsize = latent_space_dim, use_bn = True)
decoder = PAE.Decoder(input_size = (features, rec_field), channel_sizes = encdec_channels, upsamples = upsamples, zsize = latent_space_dim, use_bn = True, out_channels = 32)
pixcnn  = PAE.LGated(input_sizes = (32, features, rec_field), conditional_channels = 20, num_layers = 3, k = 7, padding = 7 // 2)

models = [encoder, decoder, pixcnn]
params = []

if torch.cuda.is_available():
    for m in models:
        m.cuda()
        params.extend(m.parameters())
    
# spdataset = TimbreDataset("G:/Projects/2022-2023/ConvDenoiser/dataset/output5/sp/", "G:/Projects/2022-2023/ConvDenoiser/dataset/output5/csp/", 256)
# sptestdataset = TimbreDataset("G:/Projects/2022-2023/ConvDenoiser/dataset/output4test/sp/", "G:/Projects/2022-2023/ConvDenoiser/dataset/output4test/csp/", 256)
spdataset = PAE.SPDataset("./traindataset/sp/", "./traindataset/csp/", segments = rec_field)
sptestdataset = PAE.SPDataset("./testdatset/sp/", "./testdatset/csp/", segments = rec_field)

spdataloader = DataLoader(spdataset, batch_size=batchsize, shuffle=True, pin_memory = False)
sptestdataloader = DataLoader(sptestdataset, batch_size=64, shuffle=False, pin_memory = False)


optimizer=torch.optim.Adam(params, lr=learning_rate)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
lossy = 0
step = 0

device_count = torch.cuda.device_count()

# if device_count > 1:
#     spdenoiser = nn.DataParallel(spdenoiser)
#     print('multiple device using :', device_count)

### Test empty tensor to check if model gives errors

In [ ]:
import models.PixelVAEDenoiser as PAE
import torch

device = 'cuda'

print(bool(encoder))
features = 60
rec_field = 256

if bool(encoder):
    # Model hyperparameters
    encdec_channels = [16, 64, 128]
    upsamples = [2, 2, 2]
    latent_space_dim = 32
    embedding_dim = 64

    encoder = PAE.Encoder(input_size = (features, rec_field), channel_sizes = encdec_channels, upsamples = upsamples, zsize = latent_space_dim, use_bn = True)
    decoder = PAE.Decoder(input_size = (features, rec_field), channel_sizes = encdec_channels, upsamples = upsamples, zsize = latent_space_dim, use_bn = True, out_channels = 32)
    pixcnn  = PAE.LGated(input_sizes = (32, features, rec_field), conditional_channels = 20, num_layers = 3, k = 7, padding = 7 // 2)

    models = [encoder, decoder, pixcnn]

    if torch.cuda.is_available():
        for m in models:
            m.cuda()
        device = 'cuda'
    else:
        device = 'cpu'

sample = torch.zeros(72, 1, features, rec_field).to("device")
testinput = torch.rand((1,1,60,256)).to(device)
testz = encoder(testinput)
testy = decoder(testz)
testx_hat = pixcnn(sample, testy)

# inny = testinput.view(testinput.size(0), -1)
# outy = inny.view(testinput.size())

print("Encoder output size:", testz.size())
print("Decoder output size:", testy.size())
print("PixelCNN output size:", testx_hat.size())

: 